# Structure manipulation

In this labs, we will brush up on manipulation with molecular 3D structures from the first labs. Then we will show how to visualize a protein (or nucleic acid) structure in a notebook and then we will proceed to 3D structure superposition.


### Bio.PDB

As we went through this package in the first labs, we will revisit it here only very briefly. Structures, irrespective of whether they are obtained from PDB or mmCIF file, are represented by the [Bio.PDB.Entity](https://biopython.org/docs/latest/api/Bio.PDB.Entity.html) module from the [Bio.PDB](https://biopython.org/docs/latest/api/Bio.PDB.html) package. A structure object has the *structure->model->chain->residue* architecture.

![SMRCA](http://biopython.org/DIST/docs/tutorial/images/smcra.png)

In [20]:
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBList import PDBList

pdbl = PDBList()
file_name=pdbl.retrieve_pdb_file("7lkr", file_format="mmCif", pdir=".")
parser = MMCIFParser()
structure = parser.get_structure("7lkr", file_name)

model = structure[0]
print(model.get_list())
chain = model['A']
print(chain.get_list()[0])
res = chain[(' ',5,' ')]
atom=res['CA']
print(atom.get_vector())
list(model.get_residues())

Structure exists: '.\7lkr.cif' 
[<Chain id=A>, <Chain id=B>]
<Residue LYS het=  resseq=5 icode= >
<Vector -4.32, 9.89, 8.89>


c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4467.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4531.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4608.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4774.
  warnings.warn(


[<Residue LYS het=  resseq=5 icode= >,
 <Residue MET het=  resseq=6 icode= >,
 <Residue ALA het=  resseq=7 icode= >,
 <Residue PHE het=  resseq=8 icode= >,
 <Residue PRO het=  resseq=9 icode= >,
 <Residue SER het=  resseq=10 icode= >,
 <Residue GLY het=  resseq=11 icode= >,
 <Residue LYS het=  resseq=12 icode= >,
 <Residue VAL het=  resseq=13 icode= >,
 <Residue GLU het=  resseq=14 icode= >,
 <Residue GLY het=  resseq=15 icode= >,
 <Residue CYS het=  resseq=16 icode= >,
 <Residue MET het=  resseq=17 icode= >,
 <Residue VAL het=  resseq=18 icode= >,
 <Residue GLN het=  resseq=19 icode= >,
 <Residue VAL het=  resseq=20 icode= >,
 <Residue THR het=  resseq=21 icode= >,
 <Residue CYS het=  resseq=22 icode= >,
 <Residue GLY het=  resseq=23 icode= >,
 <Residue THR het=  resseq=24 icode= >,
 <Residue THR het=  resseq=25 icode= >,
 <Residue THR het=  resseq=26 icode= >,
 <Residue LEU het=  resseq=27 icode= >,
 <Residue ASN het=  resseq=28 icode= >,
 <Residue GLY het=  resseq=29 icode= >,
 <Res

### ---- Begin Exercise ----
- Implement a function for protein-ligand binding sites detection. The function should find residues with an atom with within a certain distnace (parameter of the function) from any atom of any ligand (heteroatoms).
### ---- End Exercise ----

In [21]:
ligands = [r for r in structure[0].get_residues() if r.get_id()[0][0] == 'H']
ligand_atoms = []
for atoms in [l.get_atoms() for l in ligands]:
    ligand_atoms += atoms
    
threshold = 4
sites = []
for r in structure[0].get_residues():
    if r.get_id()[0][0] == ' ':
        for a in r:
            close = False
            for la in ligand_atoms:                
                if la - a < threshold:
                    close = True
                    #print(la, la.get_coord(), a, a.get_coord(), a.get_serial_number(), r.get_full_id())
                    break
            if close:
                sites.append(r)            
                break

### Structure visualization

Advanced tools for structure visualization, such as PyMOL or Chimera, exist for offline viewing or web-based plugins such as [Mol*](https://molstar.org/). As stated on Mol* GitHub, Mol* development was jointly initiated by PDBe and RCSB PDB to combine and build on the strengths of [LiteMol](https://litemol.org/) (developed by PDBe) and [NGL](https://nglviewer.org/) (developed by RCSB PDB) viewers. We are mentioning this here because NGL is (maybe soon "was" as it is not actively developed anymore) provided as a Jupyter widget and thus can be used to visualize structures in a notebook.

!!!!!! *Should you have issues with installing the NGLView, you can use the prepared [binder](https://mybinder.org/v2/gh/davidhoksza/ngl-binder/HEAD?labpath=labs.ipynb)* !!!!!!

In [30]:
import nglview as nv
nv.demo()

NGLWidget()

In [36]:
#view = nv.show_pdbid("3pqr")  # load "3pqr" from RCSB PDB and display viewer widget
view = nv.show_pdbid("7ar5")  # load "7ar5" from RCSB PDB and display viewer widget
view


NGLWidget()

In [37]:
view.remove_surface()
view.add_surface(selection="protein", opacity=0.4)

Let's now visualize the ligand binding sites we obtained from the first excercise. That can be done using the NGL's [selection language](https://nglviewer.org/ngl/api/manual/usage/selection-language.html).

In [25]:
#selection = " ".join(["{}:{} or".format(r.get_id()[1], r.parent.id) for r in binding_sites])[:-3]
selection  = '25:A or 26:A or 27:A or 28:A or 39:A or 40:A or 41:A or 42:A or 44:A or 45:A or 46:A or 48:A or 49:A or 50:A or 52:A or 54:A or 117:A or 118:A or 138:A or 139:A or 140:A or 141:A or 142:A or 143:A or 144:A or 145:A or 146:A or 147:A or 161:A or 163:A or 164:A or 165:A or 166:A or 167:A or 168:A or 170:A or 171:A or 172:A or 173:A or 181:A or 186:A or 187:A or 188:A or 189:A or 190:A or 191:A or 192:A or 25:B or 26:B or 27:B or 28:B or 39:B or 40:B or 41:B or 42:B or 44:B or 45:B or 46:B or 48:B or 49:B or 50:B or 51:B or 52:B or 54:B or 117:B or 118:B or 138:B or 139:B or 140:B or 141:B or 142:B or 143:B or 144:B or 145:B or 146:B or 147:B or 161:B or 163:B or 164:B or 165:B or 166:B or 167:B or 168:B or 169:B or 170:B or 171:B or 172:B or 173:B or 181:B or 186:B or 187:B or 188:B or 189:B or 190:B or 191:B or 192:B or 193:B or 213:B or 252:B or 253:B or 254:B or 255:B or 256:B or 257:B or 258:B or 259:B'

In [26]:
view_bp = nv.show_biopython(structure[0])
view_bp.representations = [
    {"type": "cartoon", "params": {
        "sele": "protein", "color": "residueindex"
    }},
    {"type": "ball+stick", "params": {
        "sele": "hetero"
    }},    
    {"type": "surface", "params": {
        "sele": selection,
        "opacity": 0.3,
        "color": "pink"
    }}
]
view_bp


NGLWidget()

In [8]:
view_bp.remove_surface()
#view_bp.clear_representations()
#view_bp.add_representation('surface', selection=selection, color='green')
view_bp.add_surface(selection=selection, color='green', opacity=0.3)

The widget has some many other features such as adding/updating/removing additional strucutures, representations (see the [API](http://nglviewer.org/nglview/latest/api.html)).

### Structure superposition

To superpose a pair of structures, we need a mapping between the atoms in the two structures and based on this mapping a procedure which minimizes RMSD (such as [Kabsch algorithm](https://en.wikipedia.org/wiki/Kabsch_algorithm)) can be executed. In BioPython, we can use the [Bio.PDB.Superimposer](https://biopython.org/docs/1.75/api/Bio.PDB.Superimposer.html) module. To use it, we need to specify the mapping as two sets of atoms ([set_atoms method](https://biopython.org/docs/1.75/api/Bio.PDB.Superimposer.html#Bio.PDB.Superimposer.Superimposer.set_atoms)) based on which the transformation is computed and applied ([apply method](https://biopython.org/docs/1.75/api/Bio.PDB.Superimposer.html#Bio.PDB.Superimposer.Superimposer.apply)) to one of the structures, to optimally (in terms of RMSD) map it onto the other.

Let's try this procedure on the *7lkr* and *7ar5* structures which are both structures of the SARS-CoV-2 3CL protease.

In [ ]:
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBList import PDBList

pdbl = PDBList()
file_name=pdbl.retrieve_pdb_file("7ar5", file_format="mmCif", pdir=".")


In [ ]:
parser = MMCIFParser()
structure_7ar5 = parser.get_structure("7ar5", '7ar5.cif')
structure_7lkr = parser.get_structure("7lkr", '7lkr.cif')


Let's visualize the two structures in NGL.

In [ ]:
import nglview as nv
v = nv.NGLWidget()
c1 = v.add_component(nv.BiopythonStructure(structure_7ar5[0]))
c2 = v.add_component(nv.BiopythonStructure(structure_7lkr[0]))

c1.clear_representations()
c1.add_representation('cartoon', color="blue")
c2.clear_representations()
c2.add_representation('cartoon', color="red")

v

The structures are clearly not superimposed. Morever, one of the structures is a (homo)dimer. We will need to pick one of the chains and use it to find mapping against the only chain in the other structure. To do that, we will simply use sequence alignment. But, first, the sequences of the chosen chains need to be extracted from the structure.

In [ ]:
from Bio.PDB.Polypeptide import three_to_one
def get_sequence(residues):
    return ''.join(three_to_one(r.get_resname()) for r in residues)

def get_residues(structure, chain):
    return [r for r in structure[0][chain].get_residues() if r.id[0] == ' ']

In [ ]:
res1 = get_residues(structure_7lkr, 'A')
seq1 = get_sequence(res1)
res2 = get_residues(structure_7ar5, 'A')
seq2 = get_sequence(res2)

Once we have the sequences, we can carry out sequence alignment (see the second labs for details).

In [ ]:
from Bio import Align
from Bio.Align import substitution_matrices

aligner = Align.PairwiseAligner()
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.open_gap_score = -11
aligner.end_open_gap_score = -11
aligner.extend_gap_score = -1
aligner.end_extend_gap_score = -1

In [ ]:
alns = aligner.align(seq1, seq2)
print(len(alns))

We got one alignment which looks as follows:

In [ ]:
alignment = alns[0]
print(alignment)

Now, we can use the [aligned](https://biopython.org/docs/1.75/api/Bio.Align.html?highlight=alignment#Bio.Align.PairwiseAlignment.aligned) property of the alignment that gives us ranges of residues in the respective sequences which are mapped onto each other.

In [ ]:
print(alignment.aligned)

In [ ]:
for range1, range2 in zip(alignment.aligned[0], alignment.aligned[1]):
    print(range1, range2)

From the ranges, we can enumerate indices of the residues which should be mapped.

In [ ]:
mapping = [[],[]]
for range1, range2 in zip(alns[0].aligned[0], alns[0].aligned[1]):
    mapping[0] += list(range(range1[0], range1[1]))
    mapping[1] += list(range(range2[0], range2[1]))

However, the `set_atoms` method in the `Bio.PDB.Superimposer` does not accept residue mapping but atom mapping. To do that, we extract C-alpha atoms of the mapped residues.

In [ ]:
atoms1 = []
atoms2 = []
for r, m, a in [(res1, mapping[0], atoms1), (res2, mapping[1], atoms2)]:
    for i in m:
        a.append(list(filter(lambda x: x.get_name() == 'CA', r[i].get_atoms()))[0])

Now, we have a mapping to be passed to the `Superimposer` and apply the transformation.

In [ ]:
import Bio
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(atoms1, atoms2)
super_imposer.apply(structure_7ar5.get_atoms())

After superposition, the resulting RMSD is stored in the `rms` property of the `PDB.Superimposer`:

In [ ]:
super_imposer.rms

And finally, let's visualize the superposed structures (we are using a new NGL instance).

In [ ]:
import nglview as nv
v_super = nv.NGLWidget()
c1 = v_super.add_component(nv.BiopythonStructure(structure_7ar5[0]))
c2 = v_super.add_component(nv.BiopythonStructure(structure_7lkr[0]))

c1.clear_representations()
c1.add_representation('cartoon', color="blue")
c2.clear_representations()
c2.add_representation('cartoon', color="red")
v_super

### ---- Begin Exercise ----
- Implement a function which when passed two protein structures would align all their chains, find the pair with best alignment, and align the structures based on that pair of chains. An example, where matching 'A' chains won't work are GPCR structures 7E2X and 6A94. There you need to align chain R of 7E2X with chain A or B from 6A94 to get reasonable superposition.
### ---- End Exercise ----